# Ollama + OpenAI + Python

## 1. Especificar o nome do modelo

Se tiver utilizado um modelo diferente de "phi3:mini", altere o valor na célula abaixo.  
Essa variável será usada no código ao longo do notebook.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Configurar o cliente Open AI

Normalmente, o cliente OpenAI é utilizado com OpenAI.com ou Azure OpenAI para interagir com modelos de linguagem avançados.  
No entanto, também pode ser usado com Ollama, uma vez que Ollama fornece um endpoint compatível com OpenAI em "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Gerar uma conclusão de chat

Agora podemos usar o SDK da OpenAI para gerar uma resposta para uma conversa. Este pedido deve gerar um haiku sobre gatos:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Engenharia de prompts

A primeira mensagem enviada ao modelo de linguagem é chamada de "mensagem do sistema" ou "prompt do sistema", e define as instruções gerais para o modelo.  
Pode fornecer o seu próprio prompt de sistema personalizado para orientar o modelo de linguagem a gerar resultados de uma forma diferente.  
Modifique o `SYSTEM_MESSAGE` abaixo para responder como o seu personagem favorito de um filme ou série de TV, ou obtenha inspiração para outros prompts de sistema em [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Depois de personalizar a mensagem do sistema, forneça a primeira pergunta do utilizador no `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Exemplos com poucas amostras

Outra forma de orientar um modelo de linguagem é fornecer "poucas amostras", uma sequência de perguntas/respostas de exemplo que demonstram como ele deve responder.

O exemplo abaixo tenta fazer com que um modelo de linguagem aja como um assistente de ensino, fornecendo alguns exemplos de perguntas e respostas que um assistente de ensino poderia dar, e depois apresenta ao modelo uma pergunta que um estudante poderia fazer.

Experimente primeiro e, em seguida, modifique o `SYSTEM_MESSAGE`, `EXAMPLES` e `USER_MESSAGE` para um novo cenário.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Geração Aumentada por Recuperação

RAG (Geração Aumentada por Recuperação) é uma técnica que permite a um modelo de linguagem responder a perguntas de forma precisa para um determinado domínio, primeiro recuperando informações relevantes de uma fonte de conhecimento e, em seguida, gerando uma resposta com base nessas informações.

Fornecemos um ficheiro CSV local com dados sobre carros híbridos. O código abaixo lê o ficheiro CSV, procura correspondências com a pergunta do utilizador e, em seguida, gera uma resposta com base nas informações encontradas. Note que isto demorará mais tempo do que qualquer um dos exemplos anteriores, pois envia mais dados para o modelo. Se notar que a resposta ainda não está fundamentada nos dados, pode tentar engenharia de sistemas ou experimentar outros modelos. Geralmente, o RAG é mais eficaz com modelos maiores ou com versões ajustadas de SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante ter em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
